In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# 현재경로 확인
os.getcwd()

'C:\\Users\\Jang\\python_machine_deep_learning\\7. Ensemble learning'

In [3]:
# 데이터 불러오기
data = pd.read_csv("./data/kc_house_data.csv") 
data.head() # 데이터 확인

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [4]:
'''
id: 집 고유아이디
date: 집이 팔린 날짜 
price: 집 가격 (타겟변수)
bedrooms: 주택 당 침실 개수
bathrooms: 주택 당 화장실 개수
floors: 전체 층 개수
waterfront: 해변이 보이는지 (0, 1)
condition: 집 청소상태 (1~5)
grade: King County grading system 으로 인한 평점 (1~13)
yr_built: 집이 지어진 년도
yr_renovated: 집이 리모델링 된 년도
zipcode: 우편번호
lat: 위도
long: 경도
'''

'\nid: 집 고유아이디\ndate: 집이 팔린 날짜 \nprice: 집 가격 (타겟변수)\nbedrooms: 주택 당 침실 개수\nbathrooms: 주택 당 화장실 개수\nfloors: 전체 층 개수\nwaterfront: 해변이 보이는지 (0, 1)\ncondition: 집 청소상태 (1~5)\ngrade: King County grading system 으로 인한 평점 (1~13)\nyr_built: 집이 지어진 년도\nyr_renovated: 집이 리모델링 된 년도\nzipcode: 우편번호\nlat: 위도\nlong: 경도\n'

In [5]:
nCar = data.shape[0] # 데이터 개수
nVar = data.shape[1] # 변수 개수
print('nCar: %d' % nCar, 'nVar: %d' % nVar )

nCar: 21613 nVar: 14


In [6]:
data.shape

(21613, 14)

## 의미가 없다고 판단되는 변수 제거

In [7]:
data = data.drop(['id', 'date', 'zipcode', 'lat', 'long'], axis = 1) # id, date, zipcode, lat, long  제거

## 범주형 변수를 이진형 변수로 변환
- 범주형 변수는 waterfront 컬럼 뿐이며, 이진 분류이기 때문에 0, 1로 표현한다.
- 데이터에서 0, 1로 표현되어 있으므로 과정 생략

## 설명변수와 타겟변수를 분리, 학습데이터와 평가데이터 분리

In [8]:
feature_columns = list(data.columns.difference(['price'])) # Price를 제외한 모든 행
X = data[feature_columns]
y = data['price']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state = 42) # 학습데이터와 평가데이터의 비율을 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) # 데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


# 1. regression model에 bagging 적용 안한것과 한것의 차이

## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증 (Stats_Models 이용)

In [9]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

sm_train_x = sm.add_constant(train_x, has_constant="add")
sm_model = sm.OLS(train_y, sm_train_x)
fitted_sm_model = sm_model.fit()
fitted_sm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2776.
Date:                Sun, 20 Sep 2020   Prob (F-statistic):               0.00
Time:                        20:21:10   Log-Likelihood:            -2.0826e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.186e+06   1.73e+05     41.548      0.000    6.85e+06    7.52e+06
bathrooms     1.303e+05   3960.833     32.889      0.000    1.23e+05    1.38e+05
bedrooms     -2224.7910   2382.356     -0.934      0.350   -6894.497    2444.915
condition     1.641e+04   3169.013      5.178      0.000    1.02e+04    2.26e+04
floors        1946.3052   4336.838      0.449      0.654   -6554.422    1.04e+04
grade         1.956e+05   2199.540     88.924      0.000    1.91e+05       2e+05
waterfront    7.555e+05   2.26e+04     33.479      0.000    7.11e+05       8e+05
yr_built     -4300.7865     88.073    -48.832      0.000   -4473.420   -4128.153
yr_renovated    12.7325      5.043      2.525      0.012       2.847      22.618
==============================================================================
Omnibus:                    13447.374   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684794.827
Skew:                           3.763   Prob(JB):                         0.00
Kurtosis:                      54.147   Cond. No.                     1.82e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
sm_test_x = sm.add_constant(test_x, has_constant="add")
sm_model_predict = fitted_sm_model.predict(sm_test_x)

sqrt(mean_squared_error(sm_model_predict, test_y))

239804.29670858168

## Bagging 을 이용하여 선형 회귀 모형에 적합 후 평가 (for문 사용)

In [11]:
bagging_predict_result = []
print("train_x의 데이터 수:", train_x.shape[0])
for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])]
    random_data_index = np.random.choice(data_index, train_x.shape[0]) ## train_x의 인덱스중에서 train_x의 데이터수만큼 복원추출로 뽑는다
    print("복원추출했을 때 unique한 데이터 수: ", len(set(random_data_index))) ## set을 통해 unique한 데이터인덱스만 고를 수 있다
    
    sm_train_x = train_x.iloc[random_data_index]
    sm_train_x = sm.add_constant(sm_train_x, has_constant="add")
    sm_train_y = train_y.iloc[random_data_index]
    
    sm_model = sm.OLS(sm_train_y, sm_train_x)
    fitted_sm_model = sm_model.fit()
    
    pred = fitted_sm_model.predict(sm_test_x)
    bagging_predict_result.append(pred)
    
    print("MSE: ", sqrt(mean_squared_error(pred, test_y)))
    print()

train_x의 데이터 수: 15129
복원추출했을 때 unique한 데이터 수:  9534
MSE:  241385.18961266533

복원추출했을 때 unique한 데이터 수:  9613
MSE:  239707.23080231572

복원추출했을 때 unique한 데이터 수:  9607
MSE:  239902.8780735903

복원추출했을 때 unique한 데이터 수:  9530
MSE:  239542.55608655547

복원추출했을 때 unique한 데이터 수:  9525
MSE:  239669.15250576814

복원추출했을 때 unique한 데이터 수:  9587
MSE:  240113.79244983726

복원추출했을 때 unique한 데이터 수:  9538
MSE:  240265.65376426486

복원추출했을 때 unique한 데이터 수:  9639
MSE:  239668.67652540538

복원추출했을 때 unique한 데이터 수:  9525
MSE:  239754.82088075008

복원추출했을 때 unique한 데이터 수:  9596
MSE:  239694.8995337847



In [12]:
bagging_predict = [] ## 

for list2_index in range(test_x.shape[0]):  # 0 ~ 15128
    temp_predict = []
    for list_index in range(len(bagging_predict_result)):  # 0 ~ 9
        temp_predict.append(bagging_predict_result[list_index].values[list2_index])
    bagging_predict.append(np.mean(temp_predict))
    
"Ensemble 결과 mse값:", sqrt( mean_squared_error(bagging_predict, test_y))


# 지금 상황에서는, 오히려 일반 모델보다 앙상블 모델의 성능이 더 안좋아졌다
# 앙상블기법은 overfitting이 잘되는 모델들을 결합해야 성능이 좋은데,
# 이 데이터를 선형회귀모델로 만들때 overfitting이 별로 없는것으로 추정된다.


('Ensemble 결과 mse값:', 239688.51285492248)

## 학습 데이터를 선형 회귀 모형에 적합 후 평가 데이터로 검증 (Scikit-Learn 이용)

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

regression_model = LinearRegression() # 선형 회귀 모형
linear_model1 = regression_model.fit(train_x, train_y) # 학습 데이터를 선형 회귀 모형에 적합

predict1 = linear_model1.predict(test_x) # 학습된 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과

RMSE: 239804.29670858147


## Bagging 을 이용하여 선형 회귀 모형에 적합 후 평가 (Sampling 10번)

In [14]:
from sklearn.ensemble import BaggingRegressor
bagging_model = BaggingRegressor(base_estimator = regression_model, # 선형회귀모형
                                 n_estimators = 5, # 5번 샘플링  --> 위의 for문 사용시의 for _ in range(10):의 10과 같은 역할
                                 verbose = 1) # 학습 과정 표시
linear_model2 = bagging_model.fit(train_x, train_y) # 학습 진행
predict2 = linear_model2.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

RMSE: 239753.82767766988


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## Bagging 을 이용하여 선형 회귀 모형에 적합 후 평가 (Sampling 30번)

In [15]:
bagging_model2 = BaggingRegressor(base_estimator = regression_model, # 선형 회귀모형
                                  n_estimators = 30, # 30번 샘플링
                                  verbose = 1) # 학습 과정 표시
linear_model3 = bagging_model2.fit(train_x, train_y) # 학습 진행
predict3 = linear_model3.predict(test_x) # 학습된 Bagging 선형 회귀 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSE: 239885.781970098


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


# 2. 의사결정나무모형에 bagging 적용 안한것과 한것 차이

## 학습 데이터를 의사결정나무모형에 적합 후 평가 데이터로 검증

In [17]:
from sklearn.tree import DecisionTreeRegressor
decision_tree_model = DecisionTreeRegressor() # 의사결정나무 모형
tree_model1 = decision_tree_model.fit(train_x, train_y) # 학습 데이터를 의사결정나무 모형에 적합
predict1 = tree_model1.predict(test_x) # 학습된 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict1, test_y)))) # RMSE 결과

RMSE: 299002.7920847322


## Bagging 을 이용하여 의사결정나무모형에 적합 후 평가 (for문 이용)

In [21]:
bagging_predict_result = []
print("train_x의 데이터 수:", train_x.shape[0])
for _ in range(10):
    data_index = [data_index for data_index in range(train_x.shape[0])]
    random_data_index = np.random.choice(data_index, train_x.shape[0]) ## train_x의 인덱스중에서 train_x의 데이터수만큼 복원추출로 뽑는다
    print("복원추출했을 때 unique한 데이터 수: ", len(set(random_data_index))) ## set을 통해 unique한 데이터인덱스만 고를 수 있다
    
    sm_train_x = train_x.iloc[random_data_index]
    sm_train_y = train_y.iloc[random_data_index]
    
    decision_tree_model = DecisionTreeRegressor()
    tree_model = decision_tree_model.fit(sm_train_x, sm_train_y)
    predict_tree = tree_model.predict(test_x)
    bagging_predict_result.append(predict_tree)
    
    print("MSE: ", sqrt(mean_squared_error(predict_tree, test_y)))
    print()
    
    ## 각각으로 보면 overfitting때문에 linear regression보다 오히려 성능 안좋음

train_x의 데이터 수: 15129
복원추출했을 때 unique한 데이터 수:  9573
MSE:  289199.6294340458

복원추출했을 때 unique한 데이터 수:  9552
MSE:  292446.76936214796

복원추출했을 때 unique한 데이터 수:  9644
MSE:  287372.5762739883

복원추출했을 때 unique한 데이터 수:  9592
MSE:  282803.83367055195

복원추출했을 때 unique한 데이터 수:  9647
MSE:  271422.89246585825

복원추출했을 때 unique한 데이터 수:  9522
MSE:  306337.32836935256

복원추출했을 때 unique한 데이터 수:  9606
MSE:  295887.0246649136

복원추출했을 때 unique한 데이터 수:  9569
MSE:  292410.49493883085

복원추출했을 때 unique한 데이터 수:  9453
MSE:  282776.76356550626

복원추출했을 때 unique한 데이터 수:  9548
MSE:  302838.71288132475



In [24]:
bagging_predict = [] ## 

for list2_index in range(test_x.shape[0]):  # 0 ~ 15128
    temp_predict = []
    for list_index in range(len(bagging_predict_result)):  # 0 ~ 9
        temp_predict.append(bagging_predict_result[list_index][list2_index])
    bagging_predict.append(np.mean(temp_predict))
    
"Ensemble 결과 mse값:", sqrt( mean_squared_error(bagging_predict, test_y))


## 위의 10개를 평균내어 진짜 성능을 구하니, 훨씬 좋아짐

('Ensemble 결과 mse값:', 238027.50878520074)

## Bagging 을 이용하여 의사결정나무모형에 적합 후 평가 (Sampling 10번)

In [18]:
bagging_decision_tree_model1 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 5, # 5번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model2 = bagging_decision_tree_model1.fit(train_x, train_y) # 학습 진행
predict2 = tree_model2.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict2, test_y)))) # RMSE 결과

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSE: 237828.27321561766


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


## Bagging 을 이용하여 의사결정나무모형에 적합 후 평가 (Sampling 30번)

In [26]:
bagging_decision_tree_model2 = BaggingRegressor(base_estimator = decision_tree_model, # 의사결정나무 모형
                                                n_estimators = 30, # 30번 샘플링
                                                verbose = 1) # 학습 과정 표시
tree_model3 = bagging_decision_tree_model2.fit(train_x, train_y) # 학습 진행
predict3 = tree_model3.predict(test_x) # 학습된 Bagging 의사결정나무 모형으로 평가 데이터 예측
print("RMSE: {}".format(sqrt(mean_squared_error(predict3, test_y)))) # RMSE 결과

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


RMSE: 235220.39325524535


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [27]:
# n_estimators를 100, 200 처럼 많이 높인다고 무조건 좋은것은 아니다